In [1]:
import pandas as pd

# import techminer.jupyter as jpl
from techminer.analytics import load_scopus

filepath = "../data/papers/urban-agriculture.csv"
df = pd.read_csv(filepath)
df = load_scopus(df)


100%|██████████| 8/8 [00:13<00:00,  1.63s/it]


In [2]:
import ipywidgets as widgets
from ipywidgets import Layout

import techminer.analytics as tc
import techminer.plots as plt

# import matplotlib.pyplot as plt

In [3]:
COLORMAPS = [
    "Greys",
    "Purples",
    "Blues",
    "Greens",
    "Oranges",
    "Reds",
    "YlOrBr",
    "YlOrRd",
    "OrRd",
    "PuRd",
    "RdPu",
    "BuPu",
    "GnBu",
    "PuBu",
    "YlGnBu",
    "PuBuGn",
    "BuGn",
    "YlGn",
    "Pastel1",
    "Pastel2",
    "Paired",
    "Accent",
    "Dark2",
    "Set1",
    "Set2",
    "Set3",
    "tab10",
    "tab20",
    "tab20b",
    "tab20c",
]

In [4]:
COLUMNS = [
    "Author Keywords",
    "Authors",
    "Countries",
    "Country 1st",
    "Document type",
    "Index Keywords",
    "Institution 1st"
    "Institutions",
    "Keywords",
    "Source title",
]

In [5]:
def menu_descriptive_stats(x):
    return widgets.Text("TO-DO")

In [6]:
def menu_analysis_by_term_per_term_per_year(x):
    return widgets.Text("TO-DO")

In [7]:
def menu_occurrence(x):
    return widgets.Text("TO-DO")

In [8]:
def menu_co_occurrence(x):
    return widgets.Text("TO-DO")

In [9]:
def menu_autocorrelation(x):
    return widgets.Text("TO-DO")

In [10]:
def menu_correlation(x):
    return widgets.Text("TO-DO")

In [11]:
def menu_factor_analysis(x):
    return widgets.Text("TO-DO")

In [12]:
def menu_analysis_by_year(x):
    def compute(selected, plot_type, cmap):
        df = tc.summary_by_year(x)
        df = df[data[selected]]
        plot = plots[plot_type]
        output.clear_output()
        with output:
            display(plot(df, cmap=cmap))

    #
    # Options
    #
    data = {
        "Documents by Year": ["Year", "Num Documents"],
        "Cum. Documents by Year": ["Year", "Num Documents (Cum)"],
        "Times Cited by Year": ["Year", "Times Cited"],
        "Cum. Times Cited by Year": ["Year", "Times Cited (Cum)"],
        "Avg. Times Cited by Year": ["Year", "Avg. Times Cited"],
    }
    plots = {"bar": plt.bar, "barh": plt.bar}
    #
    selected = widgets.Dropdown(
        options=list(data.keys()),
        value=list(data.keys())[0],
        disable=False,
    )
    plot_type = widgets.Dropdown(
        options=["bar", "barh"], disable=False,
    )
    cmap = widgets.Dropdown(options=COLORMAPS, disable=False,)
    #
    output = widgets.Output()
    with output:
        display(
            widgets.interactive_output(
                compute, {"selected": selected, "plot_type": plot_type, "cmap": cmap}
            )
        )
    #
    left_box = widgets.VBox(
        [
            widgets.VBox([widgets.Label(value="Plot:"), selected]),
            widgets.VBox([widgets.Label(value="Plot type:"), plot_type]),
            widgets.VBox([widgets.Label(value="Colormap:"), cmap]),
        ],
        layout=Layout(height="350px", border="1px solid gray"),
    )
    right_box = widgets.VBox([output])
    return widgets.HBox([left_box, right_box])

In [13]:
def menu_analysis_by_term(x):
    def tab_term_plots():
        def compute_by_term(term, top_n, analysis_type, plot_type, cmap):
            df = tc.summary_by_term(x, term)
            if analysis_type == "Frequency":
                df = df.sort_values(
                    ["Num Documents", "Cited by", term], ascending=False
                )
                df = df[[term, "Num Documents"]].head(top_n)
            else:
                df = df.sort_values(
                    ["Cited by", "Num Documents", term], ascending=False
                )
                df = df[[term, "Cited by"]].head(top_n)
            df = df.reset_index(drop=True)
            plot = plots[plot_type]
            output.clear_output()
            with output:
                display(plot(df, cmap=cmap))

        columns = [z for z in COLUMNS if z in x.columns]
        term = widgets.Select(options=columns, ensure_option=True, disabled=False,)

        analysis_type = widgets.Dropdown(
            options=["Frequency", "Citation"], value="Frequency", disable=False,
        )
        #
        plots = {"bar": plt.bar, "barh": plt.bar}
        plot_type = widgets.Dropdown(options=list(plots.keys()), disable=False,)
        cmap = widgets.Dropdown(options=COLORMAPS, disable=False,)
        #
        top_n = widgets.IntSlider(
            value=10,
            min=10,
            max=50,
            step=1,
            disabled=False,
            continuous_update=False,
            orientation="horizontal",
            readout=True,
            readout_format="d",
        )
        #
        output = widgets.Output()
        with output:
            display(
                widgets.interactive_output(
                    compute_by_term,
                    {
                        "term": term,
                        "top_n": top_n,
                        "analysis_type": analysis_type,
                        "plot_type": plot_type,
                        "cmap": cmap,
                    },
                )
            )
        #
        left_box = widgets.VBox(
            [
                widgets.VBox([widgets.Label(value="Term to analyze:"), term]),
                widgets.VBox([widgets.Label(value="Analysis type:"), analysis_type]),
                widgets.VBox([widgets.Label(value="Top n terms:"), top_n]),
                widgets.VBox([widgets.Label(value="Plot type:"), plot_type]),
                widgets.VBox([widgets.Label(value="Colormap:"), cmap]),
            ],
            layout=Layout(height="400px", border="1px solid gray"),
        )
        right_box = widgets.VBox([output])
        return widgets.HBox([left_box, right_box])

    def tab_worldmap():
        def compute_worldmap(term, analysis_type, cmap):
            df = tc.summary_by_term(x, term)
            if analysis_type == "Frequency":
                df = df[[term, "Num Documents"]]
            else:
                df = df[[term, "Cited by"]]
            df = df.reset_index(drop=True)
            output.clear_output()
            with output:
                display(plt.worldmap(df, cmap=cmap))

        term = widgets.Select(
            options=["Countries", "Country 1st"], ensure_option=True, disabled=False,
        )
        analysis_type = widgets.Dropdown(
            options=["Frequency", "Citation"], value="Frequency", disable=False,
        )
        cmap = widgets.Dropdown(options=COLORMAPS, disable=False,)
        #
        output = widgets.Output()
        with output:
            display(
                widgets.interactive_output(
                    compute_worldmap,
                    {"term": term, "analysis_type": analysis_type, "cmap": cmap,},
                )
            )
        #
        left_box = widgets.VBox(
            [
                widgets.VBox([widgets.Label(value="Term to analyze:"), term]),
                widgets.VBox([widgets.Label(value="Analysis type:"), analysis_type]),
                widgets.VBox([widgets.Label(value="Colormap:"), cmap]),
            ],
            layout=Layout(height="400px", border="1px solid gray"),
        )
        right_box = widgets.VBox([output])
        return widgets.HBox([left_box, right_box])

    #

    tab_nest = widgets.Tab()
    tab_nest.children = [tab_term_plots(), tab_worldmap()]
    tab_nest.set_title(0, "Term analysis")
    tab_nest.set_title(1, "Worldmap")
    return tab_nest

In [14]:
def dashboard(x):
    m = [
        (menu_descriptive_stats, "Descriptive statistics"),
        (menu_analysis_by_year, "Analysis by year"),
        (menu_analysis_by_term, "Analysis by term"),
        (menu_analysis_by_term_per_year, "Analysis by term per year"),
        (menu_occurrence, "Occurrence analysis"),
        (menu_co_occurrence, "Co-occurrence analysis"),
        (menu_autocorrelation, "Autocorrelation analysis"),
        (menu_correlation, "Correlation analysis"),
        (menu_factor_analysis, "Factor analysis"),
    ]
    accordion = widgets.Accordion()
    widget_list = []
    for f, _ in m:
        widget_list.append(f(x))
    accordion.children = widget_list
    for index, t in enumerate(m):
        accordion.set_title(index, t[1])
    return accordion

In [19]:
def menu_analysis_by_term_per_year(x):
    def compute_by_term(term, top_n, analysis_type, plot_type, cmap):        
        plot = plt.heatmap if plot_type == 'Heatmap'else plt.gant
        if analysis_type == "Frequency":
            top = tc.documents_by_term(x, term).head(top_n)[term].tolist()
            matrix = tc.documents_by_term_per_year(x, term, as_matrix=True)
        else:
            top = tc.citations_by_term(x, term).head(top_n)[term].tolist()
            matrix = tc.citations_by_term_per_year(x, term, as_matrix=True)
        print(matrix.columns)
        matrix = matrix[top]
        output.clear_output()
        with output:
            if plot_type == 'Heatmap':
                display(plot(matrix, cmap=cmap))
            else:
                display(plot(matrix))
    #       
    columns = [z for z in COLUMNS if z in x.columns]
    term = widgets.Select(options=columns, ensure_option=True, disabled=False,)
    analysis_type = widgets.Dropdown(
        options=["Frequency", "Citation"], value="Frequency", disable=False,
    )
    plots = {"Heatmap": plt.heatmap, "Gant": plt.gant}
    plot_type = widgets.Dropdown(options=list(plots.keys()), disable=False,)
    cmap = widgets.Dropdown(options=COLORMAPS, disable=False,)
    #
    top_n = widgets.IntSlider(
        value=10,
        min=10,
        max=30,
        step=1,
        disabled=False,
        continuous_update=False,
        orientation="horizontal",
        readout=True,
        readout_format="d",
    )
    #
    output = widgets.Output()
    with output:
        display(
            widgets.interactive_output(
                compute_by_term,
                {
                    "term": term,
                    "top_n": top_n,
                    "analysis_type": analysis_type,
                    "plot_type": plot_type,
                    "cmap": cmap,
                },
            )
        )
    #
    left_box = widgets.VBox(
        [
            widgets.VBox([widgets.Label(value="Term to analyze:"), term]),
            widgets.VBox([widgets.Label(value="Analysis type:"), analysis_type]),
            widgets.VBox([widgets.Label(value="Top n terms:"), top_n]),
            widgets.VBox([widgets.Label(value="Plot type:"), plot_type]),
            widgets.VBox([widgets.Label(value="Colormap:"), cmap]),
        ],
        layout=Layout(height="450px", border="1px solid gray"),
    )
    right_box = widgets.VBox([output])
    return widgets.HBox([left_box, right_box])



In [20]:
dashboard(df)

Accordion(children=(Text(value='TO-DO'), HBox(children=(VBox(children=(VBox(children=(Label(value='Plot:'), Dr…

In [ ]:
df.columns